In [18]:
# Three functions to round decimals to either one tenth, hundredth or thousandth
def tenth_round(x):
    return round(x*10)/10
def hund_round(x):
    return round(x*100)/100
def thou_round(x):
    return round(x*1000)/1000

# State_dict is a dictionary of all possible states within the environment
# Based on fidelity set below, this equates to 285 states
state_dict = {}
state_dict[0]=[0,0]
for i in range(0,285):
    state_dict[i]=[0,0]

# Build the state dictionary.
counter = 0
i_div = -1.20
# 19 divisions between -1.20 and 0.6, in 0.1 increments (Car Position)
for i in range(19):
    if (i_div == 0.8):
        i_div = -1.2
    j_div = -0.07
    # 15 divisions between -0.07 and 0.07 in 0.01 increments (Car Speed)
    for j in range(15):
        if (j_div == 0.08):
            j_div = -0.07
        # Store values in the dictionary index
        state_dict[counter] = [tenth_round(i_div),hund_round(j_div)]
        #print("State {}: {}".format(counter, state_dict[counter]))
        counter += 1
                 
        j_div += 0.01
        
    i_div += 0.1    

# Method to find a state within the state_dict
# Assumes that incoming state param contains values not formatted
# Maps state values to appropriate ranges and performs linear search
def find_state(state):
    new_state = state
    new_state[0] = tenth_round(state[0])
    new_state[1] = hund_round(state[1])

    for i in range (285):
        if(state_dict[i][0] == new_state[0]):
            if(state_dict[i][1] == new_state[1]):
                return i
    # Returning -1 will flag that state was not found
    return -1

In [10]:
## Random search
import gym
import time
env = gym.make('MountainCar-v0')
env.reset()
num_episodes = 10
total_epochs = 0
for i in range(0, num_episodes):
    epochs = 0
    penalties = 0
    env.reset()
    while True:
        # action_space.sample() returns a random action
        action = env.action_space.sample()
        # Test the random action
        observation, reward, done, info = env.step(action)
        penalties += reward
        epochs += 1
        if done:
            break
    total_epochs += epochs
    print("Timesteps taken: {} with penalties: {}".format(epochs, penalties))

print(f"Results after {num_episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / num_episodes}")

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Results after 10 episodes:
Average timesteps per episode: 200.0


In [19]:
%%time
import gym
import numpy as np
import time
import random
from IPython.display import clear_output

env = gym.make('MountainCar-v0')
#Q Learning ---------------------------------------------

#285 x 2 table for storing Q values
q_table = np.zeros([285, 3])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.9

# Train the agent for 1000 (or different) episodes
for i in range(1000):
    # Mountain car resets within a finite set of params
    # Always starts at bottom of the hills
    state = env.reset()
    # Find our remapped state
    current_state = find_state(state)
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        #Assume that with high epsilon a random action will be selected more often
        if random.uniform(0, 1) < epsilon:
            action_index = np.random.randint(3, size=1) # Explore action space
            action_index = int(action_index)
        else:
            # Select the highest weighted action from the Q Table
            action_index = np.argmax(q_table[current_state]) # Exploit learned values
    
        next_state, reward, done, info = env.step(action_index)         
        new_reward = next_state[0]/0.6 # max reward at position 0.6
        future_state = find_state(next_state)
        old_value = q_table[current_state, action_index]
        future_max = np.max(q_table[future_state])
        
        # Q Learning algorithm
        new_value = old_value + alpha * (new_reward + (gamma * future_max) - old_value)

        q_table[current_state][action_index] = new_value
        current_state = future_state
        epochs += 1
    
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 900
Training finished.

Wall time: 14.6 s


In [17]:
# Debuggin - check how many rows of Q Table were updated
counter = 0
for i in range(285):
    if(q_table[i][0] != 0 or q_table[i][1] != 0 or q_table[i][2] != 0):
        counter += 1
        #print("Q-table[{}]: {} -> state_dict: {}".format(i, q_table[i], state_dict[i]))
print("Rows filled: {}".format(counter))


Rows filled: 83


In [15]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties, total_rewards = 0, 0, 0

episodes = 10

# For x episodes run the program, always selecting the Max element from the Q Table
for _ in range(episodes):
    state = env.reset()
    new_state = find_state(state)
    epochs, penalties, reward = 0, 0, 0
    episode_reward = 0
    done = False
    epochs = 0
    penalties = 0
    while not done:
        
        action = np.argmax(q_table[new_state])
        state, reward, done, info = env.step(action)
        penalties += reward
        epochs += 1

    total_epochs += epochs
    if(state[0] >= 0.4):
        total_rewards += 1
    total_epochs += epochs
    print("Timesteps taken: {} with penalties: {}".format(epochs, penalties))

print(f"Results after {num_episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / num_episodes}")

Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Timesteps taken: 200 with penalties: -200.0
Results after 10 episodes:
Average timesteps per episode: 400.0
